## 2. Preprocesado NLP

In [2]:
# Importamos librerías

import pandas as pd
import numpy as np

# Cargamos el dataframe
# Introducir en path_file la ruta donde hemos guardado el dataset de trabajo

path_file = "XXXXX/df_videogames.csv" 
df_videogames = pd.read_csv(path_file, sep=',')

# Importamos librerías

!pip install stop-words
!pip install unicodedata2
!pip install regex
from stop_words import get_stop_words
import unicodedata
import re

# Relizaremos un primer preprocesado que elimine todo carácter que no sea una letra, creamos para ello una función

def review_normalization(review):
    review = unicodedata.normalize('NFKD', review).lower().encode('ascii', errors='ignore').decode('utf-8')
    review = re.sub(' +', ' ', ' '.join([word if word.isalpha() else '' for word in review.split()])).strip()
    return review

In [4]:
# Importamos librerías

!pip install gensim
import gensim

# Realizaremos un segundo nivel de preprocesado eliminando algunas stopwords adicionales (las palabras derivadas de 'game'), creamos para ello otra función
# Aplicaremos en este segundo nivel de preprocesado un filtrado adicional de las stopwords de gensim

sw_list = get_stop_words('en')
sw_list.append(['game', 'games', 'game.'])

def remove_stopwords(review, sw_list):
    review = ' '.join([word for word in review.split() if (word not in sw_list) or (word not in gensim.parsing.preprocessing.STOPWORDS)])
    return review

In [9]:
# Importamos librerías

!pip install nltk
import nltk
nltk.download('omw-1.4')

# Realizaremos un tercer y último nivel de preprocesado en el que aplicaremos una lematización simple, creamos para ello una nueva función

lemmatizer = WordNetLemmatizer()

def lemmatization(review):
    review = ' '.join(lemmatizer.lemmatize(word).lower().strip() for word in review.split())
    return review

# Comprobamos, con un ejemplo sencillo, que la lematización funciona correctamente

lemmatizer.lemmatize('houses')

'house'

In [10]:
# Creamos la función base para realizar el preprocesado completo (que ejecutará las 3 funciones creadas anteriormente)

def process_reviews(reviews, sw_list):
    processed_reviews = []

    for review in reviews:
            review = review_normalization(review)
            review = remove_stopwords(review, sw_list)
            review = lemmatization(review)
            processed_reviews.append(review)
    return processed_reviews

In [11]:
# Ejecutamos el preprocesado completo

processed_reviews = process_reviews(df_videogames['reviewText'], sw_list)

In [12]:
# Observamos cómo ha funcionado el preprocesado

print('Review original: {}'.format(df_videogames['reviewText'].values[0]))
print('Review procesada: {}'.format(processed_reviews[0]))

Review original: This game could have been so much more.  The hack and slash action is not even fun, it just get's tedious and boring.  I have to admit the graphic's are spectacular, but that does not make this a great game.  Rune contains lot's of jumping from object to object, which will get boring after they first introduce it into the game.  The game also had no depth whatsoever.  I never just want to play it for fun, because I felt there almost is no fun in this game.  In my opinion, if you are going to get a hack and slash action game GET BLADE OF DARKNESS.
Review procesada: game much hack slash action even just tedious admit make great rune contains jumping object will get boring first introduce game also depth never just want play felt almost fun going get hack slash action game get blade


In [13]:
# Cargamos la reviews preprocesadas en el dataframe

df_videogames.loc[:,'processedreviews'] = processed_reviews

# Eliminamos posibles valores nulos derivados del preprocesado

df_videogames['processedreviews'] = df_videogames['processedreviews'].replace('', np.nan)
df_videogames = df_videogames.dropna()

# Eliminamos la variable 'reviewText', que ya no utilizaremos de cara al análisis

df_videogames = df_videogames.drop('reviewText', axis = 1)

# Llegados a este punto consideramos que el dataframe ya está preprocesado y listo para el análisis, lo guardamos en formato csv para posteriores usos
    #df_videogames.to_csv('XXXXX/df_videogames_processed.csv', index=False)

# Conclusiones

Como ya se observó en la fase anterior, el preprocesado, eliminando signos de puntuación y stopwords, ha dejado un corpus limpio, sin "ruido" y listo para el análisis. En este sentido se ha comprobado que es muy sencillo aplicar una doble eliminación de stopwords (añadimos la de gensim), así como una lematización, lo que supondrá una mayor limpieza y menor complejidad del corpus de cara a un análisis más eficiente.